In [ ]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('project-2-exploratory-data-analysis')
jovian.set_colab_id('19r_ibZhh8ba2zZIyOGTphyd4hfml26en')

# EDA on Brazilian E-Commerce Olist

### How to run the code

This tutorial is an executable [Jupyter notebook](https://jupyter.org) hosted on [Jovian](https://www.jovian.ai). You can _run_ this tutorial and experiment with the code examples in a couple of ways: *using free online resources* (recommended) or *on your computer*.

#### Option 1: Running using free online resources (1-click, recommended)

The easiest way to start executing the code is to click the **Run** button at the top of this page and select **Run on Binder**. You can also select "Run on Colab" or "Run on Kaggle", but you'll need to create an account on [Google Colab](https://colab.research.google.com) or [Kaggle](https://kaggle.com) to use these platforms.


#### Option 2: Running on your computer locally

To run the code on your computer locally, you'll need to set up [Python](https://www.python.org), download the notebook and install the required libraries. We recommend using the [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/) distribution of Python. Click the **Run** button at the top of this page, select the **Run Locally** option, and follow the instructions.

>  **Jupyter Notebooks**: This tutorial is a [Jupyter notebook](https://jupyter.org) - a document made of _cells_. Each cell can contain code written in Python or explanations in plain English. You can execute code cells and view the results, e.g., numbers, messages, graphs, tables, files, etc., instantly within the notebook. Jupyter is a powerful platform for experimentation and analysis. Don't be afraid to mess around with the code & break things - you'll learn a lot by encountering and fixing errors. You can use the "Kernel > Restart & Clear Output" menu option to clear all outputs and start again from the top.

## Introduction
This is a Brazilian ecommerce public dataset of orders made at [Olist Store]((https://olist.com/pt-br/)). The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. Its features allows viewing an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. It has also been included a geolocation dataset that relates Brazilian zip codes to lat/lng coordinates.

Olist is a Brazilian departmental store (marketplace) that operates in e-commerce segment, but is not an e-commerce itself. It operates as a SaaS (Software as a Service) technology company since 2015. It offers a marketplace solution (of e-commerce segment) to shopkeepers of all sizes (and for most segments) to increase their sales whether they have online presence or not.

## Installing libraries and dataset

### Downloading required libraries.

In [ ]:
#Run this cell again to remove pip error
!pip install jovian opendatasets plotly folium matplotlib==3.1.3 seaborn --upgrade --quiet

In [ ]:
#Importing required libraries
import jovian
import opendatasets as od
import os
import pandas as pd
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import FastMarkerCluster, HeatMap

In [ ]:
# Execute this to save new versions of the notebook
jovian.commit(project="project-2-exploratory-data-analysis")

### Downloading dataset

Creating a variable which will contain our dataset link.

In [ ]:
dataset_url ='https://www.kaggle.com/olistbr/brazilian-ecommerce'

In [ ]:
#You need to provide your kaggle credentials
od.download(dataset_url)


In [ ]:
data_dir = 'brazilian-ecommerce'

### All csv files

Lets look at all the `.csv` files which we have downloaded

In [ ]:
os.listdir(data_dir)

### Creating data frames

Creating the required data frames for *analysis*. We have been using pd.read_csv() function to convert the `.csv` file to pandas data frame.

In [ ]:
olist_customer_df = pd.read_csv(data_dir + '/olist_customers_dataset.csv')
olist_sellers_df = pd.read_csv(data_dir + '/olist_sellers_dataset.csv')
olist_geolocation_df = pd.read_csv(data_dir + '/olist_geolocation_dataset.csv')
olist_orders_df = pd.read_csv(data_dir + '/olist_orders_dataset.csv')
olist_order_items_df = pd.read_csv(data_dir + '/olist_order_items_dataset.csv')
olist_payments_df = pd.read_csv(data_dir + '/olist_order_payments_dataset.csv')
olist_reviews_df = pd.read_csv(data_dir + '/olist_order_reviews_dataset.csv')
olist_products_df = pd.read_csv(data_dir + '/olist_products_dataset.csv')
olist_translation_df = pd.read_csv(data_dir + '/product_category_name_translation.csv')

## Data Cleaning 

### Wrong city name



* Replacing são paulo with sao paulo as both define same city

In [ ]:
olist_customer_df[:3]

In [ ]:
olist_geolocation_df.replace(to_replace='são paulo',value='sao paulo',inplace=True)
olist_geolocation_df.geolocation_city.value_counts()

### Handling blank spaces

Clearing out `blank spaces` from olist_geolocation_df to get excat cities name from the dataset

In [ ]:
top_15 = olist_geolocation_df[['geolocation_city','geolocation_lat','geolocation_lng']]
y= top_15['geolocation_city'].str.strip().value_counts().head(15)
#converting it in a list
list1=y.index.to_list()
list1

### Geographical changes

Getting out a dataframe which contains **top 15** cities having most selling points and also clearing out latitude and longitude which is beyond `Brazil`

In [ ]:
#geo data of top 15 cities with there pin codes.
geo_data = olist_geolocation_df[~olist_geolocation_df['geolocation_city'].isin(list1)]
geo_data[:3]

In [ ]:
#Brazils most Northern spot is at 5 deg 16′ 27.8″ N latitude.;
geo_data = geo_data[geo_data.geolocation_lat <= 5.27438888]
#it’s most Western spot is at 73 deg, 58′ 58.19″W Long.
geo_data = geo_data[geo_data.geolocation_lng >= -73.98283055]
#It’s most southern spot is at 33 deg, 45′ 04.21″ S Latitude.
geo_data = geo_data[geo_data.geolocation_lat >= -33.75116944]
#It’s most Eastern spot is 34 deg, 47′ 35.33″ W Long.
geo_data= geo_data[geo_data.geolocation_lng <=  -34.79314722]
geo_df = geo_data.groupby(by='geolocation_city', as_index=False).min()


In [ ]:
geo_df[:3]

### Merging columns

Adding `freight value` in products df for visulaization.

In [ ]:
olist_products_df['freight_value'] = olist_order_items_df['freight_value']
olist_products_df['product_category_name_english'] = olist_translation_df['product_category_name_english']

### Changing datatype

Changing the datatype to dateandtime() and getting out year, date ,month ,week day, day name and hours. `To be used in Q & A section`

In [ ]:
olist_orders_df['order_delivered_customer_date'] = pd.to_datetime(olist_orders_df['order_delivered_customer_date'])
olist_orders_df['order_approved_at']= pd.to_datetime(olist_orders_df['order_approved_at'])
olist_orders_df['order_delivered_carrier_date']= pd.to_datetime(olist_orders_df['order_delivered_carrier_date'])
olist_orders_df['order_estimated_delivery_date'] = pd.to_datetime(olist_orders_df['order_estimated_delivery_date'])
olist_orders_df['order_purchase_timestamp'] = pd.to_datetime(olist_orders_df['order_purchase_timestamp'])

In [ ]:
# Getting values for purchase date, year and Month
olist_orders_df['order_purchase_year'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.year)
olist_orders_df['order_purchase_month'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.month)
olist_orders_df['order_purchase_month_name'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.strftime('%b'))
olist_orders_df['order_purchase_year_month'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.strftime('%Y,%m'))
olist_orders_df['order_purchase_date'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.strftime('%Y,%m,%d'))

# Getting values for purchase day, day of the week and day_name
olist_orders_df['order_purchase_day'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.day)
olist_orders_df['order_purchase_dayofweek'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.dayofweek)
olist_orders_df['order_purchase_dayofweek_name'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.strftime('%a'))

# Getting values for specific time of a day 
olist_orders_df['order_purchase_hour'] = olist_orders_df['order_purchase_timestamp'].apply(lambda x: x.hour)
hours_bins = [-0.1, 6, 12, 18, 23]
hours_labels = ['Early morning', 'Morning', 'Afternoon', 'Night']
olist_orders_df['order_purchase_time_day'] = pd.cut(olist_orders_df['order_purchase_hour'], hours_bins, labels=hours_labels)
olist_orders_df[:5]

## Reading Dataset

### Looking at dataframe

In [ ]:
# Collections for each dataset
datasets = [olist_customer_df, olist_geolocation_df, olist_orders_df, olist_order_items_df, olist_payments_df,
            olist_reviews_df, olist_products_df, olist_sellers_df, olist_translation_df]
names = ['olist_customer_df', 'olist_geolocation_df', 'olist_orders_df', 'olist_order_items_df', 'olist_payments_df',
         'olist_reviews_df', 'olist_products_df', 'olist_sellers_df','olist_translation_df']

# Creating a DataFrame with useful information about all datasets
data_info = pd.DataFrame({})
data_info['dataset'] = names
data_info['n_rows'] = [df.shape[0] for df in datasets]
data_info['n_cols'] = [df.shape[1] for df in datasets]
data_info['null_amount'] = [df.isnull().sum().sum() for df in datasets]
data_info['qty_null_columns'] = [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
data_info['null_columns'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]

data_info.style.background_gradient()

We have `9 datasets` to analyse and above info can show us number of rows, missing values and number of columns in each dataframe.

### Visualizing top 15 cities

In [ ]:
customer_city = olist_customer_df.customer_city.value_counts().head(15)
customer_city

In [ ]:
px.bar(x = customer_city.index,
       y =  customer_city.values,
       title="Top 15 city with most orders",
      labels={'y':'No. of orders',
             'x':'City Name'})

As we can visualize sao paulo has got most number of orders reaching upto **15k** over the course of 2 years.

### Visulaizing top 15 states

In [ ]:
#top 15 states based of customers
customer_state_df = olist_customer_df.customer_state.value_counts().head(15).to_frame()

In [ ]:
customer_state_df.plot(kind='barh',title='Top 15 states with most orders',figsize=(18, 8));

### Translation of cateogries 
Lets look at all the cateogries present in olist in english language. 

In [ ]:
olist_translation_df[:3]

In [ ]:
words = ' '.join((i for i in olist_translation_df.product_category_name_english.dropna().str.replace(';', ' ').str.replace(',', ' ')))

wc = WordCloud( width=800, height=400, background_color='black').generate(words)

plt.figure(figsize=(16,8))
plt.axis("off")
plt.grid(False)
plt.imshow(wc);

### Plotting pin codes on map
Lets see the selling and buying points of `sao paulo` according to zip codes.

In [ ]:
#getting first 30000 zip codes
latitude = list(geo_data['geolocation_lat'])[:30000]
longitude = list(geo_data['geolocation_lng'])[:30000]
locations = list(zip(latitude, longitude))

In [ ]:
m = folium.Map(location=[-23.54562128115268	, -46.63929204800168], zoom_start=11)
FastMarkerCluster(data=locations).add_to(m)
m

In the above graph numbers shows the amount of `zip codes` present in that location and `darker` the colour higher will be the amount of pin codes.

In [ ]:
# Grouping geolocation data for plotting a heatmap
heat_data = geo_data.groupby(by=['geolocation_lat', 'geolocation_lng'], as_index=False).count().iloc[:, :3]

# Creating a mapa using folium
map1 = folium.Map(
    location=[-15, -50], 
    zoom_start=4.0, 
    tiles='cartodbdark_matter'
)

# Plugin: HeatMap
HeatMap(
    name='Heat Map',
    data=heat_data,
    radius=10,
    max_zoom=13
).add_to(map1)

map1

By the map we showed above, we have the insight that the southeast of Brazil has the highest number of orders given through **olist** e-commerce which can be true as most devloped cities lies in that area.

## Q & A

### Q1 Payment methods
1- How many types of payment methods has been used for shopping and who got the highest percentage?

In [ ]:
payment_types_df = olist_payments_df.payment_type.value_counts().to_frame().reset_index()
payment_types_df['Percentage'] = round(payment_types_df.payment_type *100/payment_types_df.payment_type.sum(),2)
payment_types_df


In [ ]:
types = len(payment_types_df)
payment_type = payment_types_df.iloc[0]['index']
highest_percentage = payment_types_df.iloc[0]['Percentage']
count = payment_types_df.iloc[0]['payment_type']
print(f'There are {types} types of method used for shopping and {payment_type} has been used {count} times over the course of 2 years sharing {highest_percentage}% among other types of payment.')

In [ ]:
plt.figure(figsize=(12, 6))
sns.set(font_scale=20)
sns.set_theme(style="darkgrid")
sns.barplot(x='index',y='Percentage',data=payment_types_df);

plt.legend(title='Payment type Distribution');
plt.xlabel('Method type', fontsize=16);
plt.ylabel('Percentage', fontsize=16);

There are 5 types of method used for shopping and credit_card has been used 76795 times over the course of 2 years sharing **73.92%** among other types of payment.<br>
Boleto is bank wallet type of mode and we can say that **olist** provides better **voucher payments**  as it has got more amount of share than **debit card**

### Q2 Cateorgies
2- Show the number of categories present in olist and how many items are present in that cateogry?

In [ ]:
category_name = olist_products_df.product_category_name.value_counts().to_frame()
category_name_df1 = category_name[:15]

In [ ]:
fig = px.histogram(data_frame=category_name_df1,
           x='product_category_name',y= category_name_df1.index)

fig.update_layout(title="Top 15 types of category",
                    xaxis_title="Product count",
                    yaxis_title="Category Name",
                    plot_bgcolor='#000000',
    font=dict(
        family="Times New Roman",
        size=14,
        color="#8B0000"
    )
                 )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Above graph shows **top 15** category types and we can visualize from the graph `cama_mesa_banho` which means `bed, table and bath` has got the most amount of products touching around 3000 different types of products in it.

In [ ]:
category_name_df2 = category_name[-15:]
fig = px.histogram(data_frame=category_name_df2,
           x='product_category_name',y= category_name_df2.index)

fig.update_layout(title="Bottom 15 types of category",
                    xaxis_title="Product count",
                    yaxis_title="Category Name",
                    plot_bgcolor='#000000',
    font=dict(
        family="Times New Roman",
        size=14,
        color="#8B0000"
    )
                 )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Here we are looking at **least 15** and we can visulize that **CD and DVDs** category got just one product in it, which leads to a conclusion that it is not for music enthusiasts.

### Q3 Freight values
3- Which cateogry cost the most transportation cost and why? 

A freight value is a price at which a certain cargo is delivered from one point to another. The price depends on the form of the cargo, the mode of transport, the weight of the cargo and the distance to the delivery destination.

In [ ]:
olist_products_df[:3]

In [ ]:
plt.figure(figsize = (16,10))
sns.heatmap(olist_products_df.pivot_table(index=['product_category_name_english'],values=['freight_value'],aggfunc='mean'));

As we can visualize from the heatmap **home_counstruction** freight value goes above `70` and as we know  Freight value has weight as one of a crietria and construction based materials has got higher weights resulting in greater freight value.

### Q4 Selling points
4- List top 15 cities having most selling points.


In [ ]:
seller_city_data = olist_sellers_df.seller_city.value_counts().head(15)
seller_city_df = seller_city_data.to_frame()

In [ ]:
fig = px.bar(seller_city_df, 
             x="seller_city", 
             y=seller_city_df.index, 
             color="seller_city")
fig.update_layout(title="Top 15 cities having most selling points",
                xaxis_title="Number of orders",
                yaxis_title="City name",
                legend_title="length",
                plot_bgcolor='#000000',
    font=dict(
        family="Garamond",
        size=17,
        color="#8B0000"
    )
                 )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_yaxes(categoryorder="total ascending")
fig.show()

**Sao paulo** has got most number of selling point which is around `700` which can be true as it is financial capital of **Brazil** and of **Latin America** as a whole.

### Q5 Order status
5- Whats is the status of orders around the course of 2 years?

In [ ]:
olist_orders_df[:3]

In [ ]:
all_count = olist_orders_df.order_status.value_counts().sum()
status = olist_orders_df.order_status.value_counts().to_frame()
status['Percentage'] = round((olist_orders_df.order_status.value_counts() / all_count) *100,3)

In [ ]:
fig = px.pie(status,
             values='Percentage',
             names=status.index,
             color_discrete_sequence=px.colors.sequential.RdBu,
             title='Order status',
             hover_data=['order_status'])
fig.update_traces( textinfo='percent+label')
fig.update_layout(uniformtext_minsize=18)
fig.show()

**Olist** has successful delivery percentage around `97%` which states that it has good delivery service which is crucial for any e-commerce brand.

### Q6 Delivery-1
6. List out how many days it take the orders to get delivered.


In [ ]:
olist_orders_df[:5]

In [ ]:
delivery_time = (olist_orders_df['order_delivered_customer_date'].dt.date - olist_orders_df['order_purchase_timestamp'].dt.date ).value_counts().rename('Count')
delivery_time_df = delivery_time.to_frame().reset_index()
delivery_time_df = delivery_time_df[:30]


In [ ]:
fig=px.bar(delivery_time_df,y=delivery_time_df['index'].dt.days,x='Count',orientation='h')
fig.update_yaxes(dtick=1)
fig.update_layout(title="Delivery Days ",
                yaxis_title="Number of month day on which order got delivered",
                xaxis_title="Number of orders on that day",              
    font=dict(
        family="Helvetica",
        size=17,
        color="#2F4F4F"
    )
                 )
fig.show()

It seems most of the orders got *delivered* in `7 days = 7000 ` after the time of purchase from olist and around 700 orders got delivered at same day of purchase. 

### Q7 Delivery-2
7- Visualizing orders over every month from our dataset. Can you see some trend?

In [ ]:
olist_orders_df= olist_orders_df.sort_values('order_purchase_date',axis=0)

In [ ]:
px.scatter(olist_orders_df,x='order_purchase_year_month',y='order_purchase_day')

Above scatter plot shows that E-commerce on Brazil really has a growing from 2016 with the time. We can see some peaks at specific months, but after 8 month of 2018 the trends needs to go down.

### Q8 Subplots on days
8- Show the shopping patterns of olist customers.

In [ ]:
#converting in data and time datatyoe and getting out day name for visulaization
olist_orders_df['order_delivered_customer_date'] =  pd.to_datetime(olist_orders_df['order_delivered_customer_date'], format ="%Y.%m.%d")
olist_orders_df['order_purchase_timestamp'] =  pd.to_datetime(olist_orders_df['order_purchase_timestamp'], format ="%Y.%m.%d")
olist_orders_df['order_approved_at'] =  pd.to_datetime(olist_orders_df['order_approved_at'], format ="%Y.%m.%d")
olist_orders_df['order_estimated_delivery_date'] =  pd.to_datetime(olist_orders_df['order_estimated_delivery_date'], format ="%Y.%m.%d")
olist_orders_df['order_delivered_carrier_date'] =  pd.to_datetime(olist_orders_df['order_delivered_carrier_date'], format ="%Y.%m.%d")

olist_orders_df.info()

In [ ]:
graph1 = olist_orders_df['order_purchase_timestamp'].dt.day_name().value_counts()
graph2 = olist_orders_df['order_approved_at'].dt.day_name().value_counts()
graph3 = olist_orders_df['order_delivered_customer_date'].dt.day_name().value_counts()
graph4 = olist_orders_df['order_estimated_delivery_date'].dt.day_name().value_counts()

In [ ]:
#plotting all subplots 
f1 = px.bar(x=graph1.index,y=graph1.values)
f2= px.bar(x=graph2.index,y=graph2.values)
f3= px.bar(x=graph3.index,y=graph3.values)
f4= px.bar(x=graph4.index,y=graph4.values)

In [ ]:
#getting the traces out of it.
trace1 = f1['data'][0]
trace2 = f2['data'][0]
trace3 = f3['data'][0]
trace4 = f4['data'][0]

In [ ]:
#making a subplot having two rows and two columns.
fig = make_subplots(rows = 2, cols =2 , shared_xaxes = False,subplot_titles=("Order Purchase ", "Order Delivery ", "Order Approved ", "Order Estimated Delivery "))

In [ ]:
#adding traces in the figure
fig.add_trace(trace1, row = 1, col = 1)
fig.add_trace(trace2, row = 2, col = 1)
fig.add_trace(trace3, row = 1, col = 2)
fig.add_trace(trace4, row = 2, col = 2)
fig.update_layout(height=800, width=800, title_text="Stacked Subplots",plot_bgcolor='#F5FFFA')

fig.update_xaxes(title_text="Days")
fig.update_yaxes(title_text='Count')

fig.show()

Insights from subplots.

*   `Order purchase` graph shows us that online shopping has be done throught the week days rather than weekend as it required minimal effort.

*   `Order Approved` graph shows us the at what days order got approved on olist and by comparing them with first graph we can conclude that on **Mondays** most people buy there stuff and it got approved on very next day on **Tuesday** which can be seen in second graph.

*   `Order Delivery` graph shows us the day of delivery and near about **19k** orders get delivered on first day of week while just **7000** items get delivered on weekends.

*   `Order Estimated delivery` this graph shows us that **Olist e-commerce** skips to give estimated delivery on weekends as weekend tends to be resting day they prefer not to disturb there customers.




### Q9 Shopping hours
9- At what time of the day does people of Brazil tends to do shopping?

In [ ]:
hours_count = olist_orders_df.order_purchase_time_day.value_counts()

In [ ]:
plt.figure(figsize=(12,10))
plt.title('Dis')
plt.pie(hours_count, labels=hours_count.index, autopct='%1.1f%%', startangle=180);

As we can see from the pie chart that **olist** recieves most of the orders at `afternoon` touching nearly `40%` and least at `early morning` which is pratically true.

## CONCLUSION


*   We have gone trough a extensive journey for understanding the data and plotting some useful charts to clarify the concepts and get insights from data.

*   Credict card is most used for payments in olist which is true because credit card company provides most amount of offers for there customers.

*   There are 74 different cateogry present in olist from where Brazilians can shop.

*   It seems transportation cost(freight value) in olist is reasonable getting max upto 70 brazilian real.

*  olist has nearly 97% successful delivery rate which shows there commitment towards the order and making this amount of successful deliveries will help them grow in near future.





## FUTURE WORK
- In near future we can look at all the reviews and comments and will try to analyze what people thinks about olist store.

- We can also use this dataset to map differently on graph as we have geo location of every pincode we can explore more into it.

- One more thing we can do is to list pricing of the products and tends to compare it with freight values which will be nect step to analyze transportation costs.

- Last but not the least we can try for a new olist dataset and comparing it will give us result whether its truly a geniune datset.

## REFERENCES

- [Aakash N S. Analyzing Tabular Data with Pandas, 2021](https://jovian.ai/aakashns/python-pandas-data-analysis)

- [Aakash N S. Data Visualization using Python Matplotlib and Seaborn](https://jovian.ai/aakashns/python-matplotlib-data-visualization) 

- [Aakash N S. Advanced Data Analysis Techniques with Python & Pandas](https://jovian.ai/aakashns/advanced-data-analysis-pandas)

- [Aakash N S. Interactive Visualization with Plotly](https://jovian.ai/aakashns/interactive-visualization-plotly)

- [Plotly Documentation](https://plotly.com/python/)

- [Aakash N S. Plotly-line-chart](https://jovian.ai/aakashns/plotly-line-chart)

- [Jovian EDA from strach](https://youtu.be/kLDTbavcmd0)